In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import numpy as np
import tensorflow as tf

np.set_printoptions(precision=3, suppress=True)

In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [4]:
train_file_path

'/root/.keras/datasets/train.csv'

In [5]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [6]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [7]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))
    print("{:20s}: {}".format("label",label.numpy()))
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [28. 28. 28. 51. 40.]
n_siblings_spouses  : [0 0 0 0 1]
parch               : [0 0 0 0 0]
fare                : [221.779   7.75   56.496   8.05    9.475]
class               : [b'First' b'Third' b'Third' b'Third' b'Third']
deck                : [b'C' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'n']
label               : [0 1 1 0 0]


In [8]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [9]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [10]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
class               : [b'Third' b'Second' b'Third' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'C' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Cherbourg' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']
numeric             : [[28.     0.     0.     7.896]
 [18.     0.     2.    13.   ]
 [22.     0.     0.     7.229]
 [30.     0.     0.    27.75 ]
 [ 9.     4.     2.    31.275]]
label               : [0 1 0 0 0]


In [11]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [12]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [13]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [14]:
# See what you just created.
import functools
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fe5f006d2f0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [15]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479, -0.482],
       [-0.13 , -0.474, -0.479, -0.488],
       [-0.13 , -0.474, -0.479, -0.392],
       [-0.13 ,  2.132,  0.782, -0.163],
       [ 0.109,  0.395, -0.479,  0.414]], dtype=float32)

In [16]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])


[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [17]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy())

[[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.    -0.13  -0.474
  -0.479 -0.482  0.     1.   ]
 [ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     1.    -0.13  -0.474
  -0.479 -0.488  0.     1.   ]
 [ 1.     0.     0.     1.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.    -0.13  -0.474
  -0.479 -0.392  1.     0.   ]
 [ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.    -0.13   2.132
   0.782 -0.163  0.     1.   ]
 [ 0.     1.     1.     0.     0.     0.     1.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.109  0.395
  -0.479  0.414  1.     0.   ]]


In [18]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [19]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 1s 12ms/step - loss: 0.5053 - accuracy: 0.7273
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4233 - accuracy: 0.8182
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4009 - accuracy: 0.8230
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3926 - accuracy: 0.8373
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3759 - accuracy: 0.8278
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3799 - accuracy: 0.8309
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3636 - accuracy: 0.8421
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3550 - accuracy: 0.8469
Epoch 9/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3526 - accuracy: 0.8469
Epoch 10/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3457 - accuracy: 0.850

In [20]:
test_loss, test_accuracy = model.evaluate(test_data)


     53/Unknown - 0s 8ms/step - loss: 0.4384 - accuracy: 0.8447